In [1]:
import torch 
import torch.nn as nn
import numpy as np
from torch.utils import data
import time
from DNN import Seq2Seq, LWRDataset, LWRDataset_res, train_LWR, eval_LWR, test_LWR, train_hybrid_LWR, eval_hybrid_LWR, test_hybrid_LWR 
from AutoODE import LWR_batch_version, LWR_seq2seq 
device = "cuda"

In [3]:
class LWR_seq2seq_joint(nn.Module): 
    def __init__(self): 
        super(LWR_seq2seq_joint, self).__init__()
        self.LWR_model = model = LWR_batch_version(nx, 300, 6, kj, vf, tskip, plm = False, plm_vf = False, 
                          initial={}, boundary={}, fix_vf=False, parstep=1).to(device).to(device) 

        self.residual_model = Seq2Seq(input_dim = 231, hidden_dim = 512, output_dim = 231, num_layers = 1).to(device)
    
    def forward(self, xi, x, initial, boundary, tsteps): 
        pred1 = self.LWR_model(xi, initial[:, 0], boundary[:, 0], tsteps) # first 12
        pred = self.LWR_model(xi, initial[:, 1], boundary[:, 1], tsteps) # last 12
        res = x[:, :, 1:13, 1:] - pred1

        residual = self.residual_model(res.float(), 12) # last 12

#         pred = pred2.detach().clone()
        pred += residual # residual[:, :, 1:] 
        
        return pred.double() 

### Load Data 

In [ ]:
# training_data = torch.load("data_LWR/residual/training_time.pt").double()
# training_initial = torch.load("data_LWR/residual/training_time_initial.pt").double()
# training_boundary = torch.load("data_LWR/residual/training_time_boundary.pt").double()
# x_train = torch.load("data_LWR/residual/x_train.pt").long()
# test_data = torch.load("data_LWR/residual/test_time.pt").double()
# test_initial = torch.load("data_LWR/residual/test_time_initial.pt").double()
# test_boundary = torch.load("data_LWR/residual/test_time_boundary.pt").double()
# x_test = torch.load("data_LWR/residual/x_test.pt").long() 

In [ ]:
# training_data = torch.load("data_LWR/space/training_time.pt").double()
# training_initial = torch.load("data_LWR/space/training_time_initial.pt").double()
# training_boundary = torch.load("data_LWR/space/training_time_boundary.pt").double() 
# test_data = torch.load("data_LWR/space/test_time.pt").double()
# test_initial = torch.load("data_LWR/space/test_time_initial.pt").double()
# test_boundary = torch.load("data_LWR/space/test_time_boundary.pt").double() 
# x_train = torch.load("data_LWR/space/x_train.pt").long()
# x_test = torch.load("data_LWR/space/x_test.pt").long() 

In [4]:
data1 = torch.load("../pems_I5_S_correct.pt") 

In [5]:
xi = [0,  45,  56,  75,  81,  86,  89,  95, 100, 105, 109, 112, 117,
       124, 128, 133, 137, 141, 146, 149, 152, 158, 163, 167, 171, 174,
       180, 186, 192, 197, 200, 205, 207, 210, 211, 213, 214, 228, 231,
       237, 240, 242, 251, 254, 258, 262, 266, 270, 277, 279, 282, 283,
       286, 288, 291, 294, 296, 298, 300, 303, 308, 310, 315, 317, 320,
       322, 327, 338, 342, 345, 352, 356, 359, 362, 366, 368, 374, 379] 

In [6]:
import numpy as np
#nx=350  # 
nx=380 
#kj needs to be larger than k for the solution to be stable 
# kj = np.ones((nx,), dtype=float) * 0.6
# kj = (kj - mean[0].numpy()) / std[0].numpy()
# kj = (kj - y_min[0].numpy()) / (y_max[0].numpy() - y_min[0].numpy()) # normalize

#characteristic velocity (m/s), corresponds to roughly 120 km/h
# vf = np.ones((nx,), dtype=float) * 38
# vf = (vf - mean[2].numpy()) / std[2].numpy()
# vf = (vf - y_min[2].numpy()) / (y_max[2].numpy() - y_min[2].numpy()) # normalize

dx=300.

## change the timestep to dt = 1, previously dt = 6 with 7 mins runtime
dt=6
#need an output every 5 mins (300 s), so tskip = 3 with dt = 3s
tskip=50
#nt=int(3600*6/6 - 50)
#nt=7099 #6 hours (times 3600 s/hour divided by dt=3s)
nto=13 + 1
#nt=int(3600*nto/12/6/dt - tskip)
dtobs=300
nt=int((dtobs*nto)/dt - tskip) 

nto=13 + 1
#nt=int(3600*nto/12/6/dt - tskip)
dtobs=300
nt_test=int((dtobs*nto)/dt - tskip)
print(dt, nt, nt_test)

6 650 650


In [7]:
from scipy.interpolate import interp1d

def interpolate_initial(var, xi, t0=0): 
    IM_q=interp1d(np.array(xi) * dx, var[1, :].numpy(), bounds_error=False, 
                fill_value=(var[1, 0], var[1, -1]), kind='linear') 
    IM_u=interp1d(np.array(xi) * dx, var[2, :].numpy(), bounds_error=False, 
                fill_value=(var[2, 0], var[2, -1]), kind='linear') 
    
    x=np.linspace(0, (nx-1) * dx, nx) 
    u = IM_u(x)
    q = IM_q(x)
    k = q / u
    initial = np.stack((k, q, u))
    return torch.tensor(initial)

In [8]:
ti=np.arange(0, (nt-1)*dt, tskip*dt )
t=np.linspace(0, (nt-1)*dt, nt) 
ti_test=np.arange(0, (nt_test-1)*dt, tskip*dt )
t_test = np.linspace(0, (nt_test-1)*dt, nt_test)

def interpolate_boundary(var, ti, t): 
    IM_q=interp1d(np.array(ti), var[1, :, 0].numpy(), bounds_error=False,
             fill_value=(var[1, 0, 0], var[1, -1, 0]), kind='linear')
    IM_u=interp1d(np.array(ti), var[2, :, 0].numpy(), bounds_error=False,
             fill_value=(var[2, 0, 0], var[2, -1, 0]), kind='linear')
    u = IM_u(t)
    q = IM_q(t)
    k = q / u
    boundary = np.stack((k, q, u)) 
    return torch.tensor(boundary)

In [9]:
def generate_data_lwr_res(data): 
#     interpolated = []
#     for i in range(data.shape[1]): 
#         interpolated_d = interpolate_initial(data[:, i, :], xi, t0=0) 
#         interpolated.append(interpolated_d) 
#     interpolated_data = torch.stack(interpolated) 
#     interpolated_data = interpolated_data.permute(1, 0, 2)
#     print(interpolated_data.shape) 
    
    training_set = [] 
    test_set = [] 
    initial_train = []
    boundary_train = []
    initial_test = []
    boundary_test = []
    x_train = []
    x_test = []
    training_size = int((data.shape[1] - 25) * 0.8) 
    
    for i in range(data.shape[1] - 25): 
        if i < training_size: 
            x_train.append(torch.tensor(xi))
            training_set.append(data[:, i:i+25, :])
            initial1 = interpolate_initial(data[:, i, :], xi, t0=0)
            boundary1 = interpolate_boundary(data[:, i:i+13, :], ti, t) 
            initial2 = interpolate_initial(data[:, i+12, :], xi, t0=0)
            boundary2 = interpolate_boundary(data[:, i+12:i+25, :], ti, t)
            initial = torch.stack([initial1, initial2])
            boundary = torch.stack([boundary1, boundary2])
            initial_train.append(initial)
            boundary_train.append(boundary)
        else: 
            x_test.append(torch.tensor(xi))
            test_set.append(data[:, i:i+25, :])
            initial1 = interpolate_initial(data[:, i, :], xi, t0=0)
            boundary1 = interpolate_boundary(data[:, i:i+13, :], ti, t) 
            initial2 = interpolate_initial(data[:, i+12, :], xi, t0=0)
            boundary2 = interpolate_boundary(data[:, i+12:i+25, :], ti, t)
            initial = torch.stack([initial1, initial2])
            boundary = torch.stack([boundary1, boundary2])
            initial_test.append(initial)
            boundary_test.append(boundary) 
    
    return torch.stack(x_train), torch.stack(x_test), torch.stack(training_set), torch.stack(test_set), torch.stack(initial_train), torch.stack(initial_test), torch.stack(boundary_train), torch.stack(boundary_test)  

x_train, x_test, training_set, test_set, initial_train, initial_test, boundary_train, boundary_test = generate_data_lwr_res(data1) 

In [ ]:
sensor_idx = list(np.arange(380))

def generate_data_lwr_date(data): 
    interpolated = []
    for i in range(data.shape[1]): 
        interpolated_d = interpolate_initial(data[:, i, :], xi, t0=0) 
        interpolated.append(interpolated_d) 
    interpolated_data = torch.stack(interpolated) 
    interpolated_data = interpolated_data.permute(1, 0, 2)
    print(interpolated_data.shape) 

    training_set = [] 
    test_set = [] 
    initial_train = []
    boundary_train = []
    initial_test = []
    boundary_test = []
    x_train = []
    x_test = [] 
    training_size = int((data.shape[1] - 13) * 0.8) 
    
    for i in range(data.shape[1] - 13): 
        if i < training_size: 
            x_train.append(torch.tensor(sensor_idx))
            training_set.append(interpolated_data[:, i:i+4, :]) 
            initial = interpolate_initial(data[:, i, :], xi, t0=0)
            boundary = interpolate_boundary(data[:, i:i+4, :], ti, t) 
            initial_train.append(initial)
            boundary_train.append(boundary)
        else: 
            x_test.append(torch.tensor(xi)) 
            test_set.append(data[:, i:i+13, :]) 
            initial = interpolate_initial(data[:, i, :], xi, t0=0)
            boundary = interpolate_boundary(data[:, i:i+13, :], ti_test, t_test)
            initial_test.append(initial)
            boundary_test.append(boundary)
    return torch.stack(x_train), torch.stack(x_test), torch.stack(training_set), torch.stack(test_set), torch.stack(initial_train), torch.stack(initial_test), torch.stack(boundary_train), torch.stack(boundary_test)  

x_train, x_test, training_set, test_set, initial_train, initial_test, boundary_train, boundary_test = generate_data_lwr_date(data1) 

In [ ]:
# torch.save(time_train, "data_LWR/time_train.pt") 
# torch.save(time_test, "data_LWR/time_test.pt")
# torch.save(training_set1, "data_LWR/training_time.pt")
# torch.save(test_set1, "data_LWR/test_time.pt")
# torch.save(initial_train, "data_LWR/training_time_initial.pt")
# torch.save(initial_test, "data_LWR/test_time_initial.pt")
# torch.save(boundary_train, "data_LWR/training_time_boundary.pt")
# torch.save(boundary_test, "data_LWR/test_time_boundary.pt")
# torch.save(x_train, "data_LWR/x_train.pt")
# torch.save(x_test, "data_LWR/x_test.pt")

In [10]:
training_set = LWRDataset_res(x_train, training_set, initial_train, boundary_train)
test_set = LWRDataset_res(x_test, test_set, initial_test, boundary_test) 
# training_set = LWRDataset_res(x_train, training_data, training_initial, training_boundary)
# test_set = LWRDataset_res(x_test, test_data, test_initial, test_boundary)
training_set, val_set = data.random_split(training_set, [int(len(training_set) * 0.875), int(len(training_set) - int(len(training_set) * 0.875))])

In [11]:
train_loader = data.DataLoader(training_set, batch_size = 256, shuffle = True, num_workers=0, pin_memory=True)
val_loader = data.DataLoader(val_set, batch_size = 512, shuffle = False, num_workers=0, pin_memory=True)
test_loader = data.DataLoader(test_set, batch_size = 512, shuffle = False, num_workers=0, pin_memory=True) 

### Hyperparameter 

In [ ]:
# trial_1 = torch.load("AutoODE_result/time/best_AutoODE1.pt") 
# LWR_model = trial_1["model"] 
# LWR_model.to(device) 

In [ ]:
# for p in LWR_model.parameters(): 
#     p.requires_grad = False 

# print(sum(p.numel() for p in LWR_model.parameters() if p.requires_grad)) 

In [15]:
from scipy.interpolate import interp1d 

def calibrate(data): 
    # assume the linear relation u = a * k + b 
    # cross all sensors / at each sensor 
#     print(data.shape)
    kj = data[0, :42613].max(dim = 0).values
    vf = data[2, :42613].max(dim = 0).values 
    IM_kj=interp1d(np.array(xi) * dx, kj.numpy(), bounds_error=False, 
                fill_value=(kj[0], kj[-1]), kind='linear') 
    IM_vf=interp1d(np.array(xi) * dx, vf.numpy(), bounds_error=False, 
                fill_value=(vf[0], vf[-1]), kind='linear')

    x=np.linspace(0, (nx-1) * dx, nx) 
    kj = IM_kj(x) 
    vf = IM_vf(x) 
#     kj = (kj - y_min[0].numpy()) / (y_max[0].numpy() - y_min[0].numpy()) 
#     vf = (vf - y_min[2].numpy()) / (y_max[2].numpy() - y_min[2].numpy())
#     k_mean = data[0].mean(dim = 0)
#     u_mean = data[2].mean(dim = 0) 
#     k_m = data[0] - k_mean
    
#     b = ((k_m) * (data[2] - u_mean)).sum(dim = 0) / (k_m * k_m).sum(dim = 0)
# #     print(b.shape)
# #     a = u_mean - b * k_mean # vf
#     kj = -(u_max / b)
#     return kj, a   
    return kj * 2, np.ceil(vf) 

In [16]:
data2 = torch.load("../pems_I5_S_correct.pt") 
kj, vf = calibrate(data2) 
# kj, vf 

In [17]:
steps=torch.tensor(np.linspace(0, nt, nt), requires_grad=True) 

In [18]:
steps_test=torch.tensor(np.linspace(0, nt_test, nt_test), requires_grad=True) 

In [25]:
device = "cuda" 
model = LWR_seq2seq_joint().to(device) 
# model = LWR_batch_version(nx, 300, 6, kj, vf, tskip, plm = False, plm_vf = False, 
#                           initial={}, boundary={}, fix_vf=False, parstep=1).to(device) 

In [26]:
name = "hybrid_LWR_joint"
learning_rate = 0.002
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 1, gamma=0.95)
criterion = nn.MSELoss()
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

best_loss = 100   
train_losses = []
val_losses = []
tsteps = steps.shape[0]
num_epoch = 100 
trial = 1 

3170783


### Training and Evaluation 

In [ ]:
import time 

for epoch in range(1, num_epoch + 1): 
    start = time.time()
    train_loss = train_LWR(model, train_loader, optimizer, criterion, tsteps)[-1]
    train_losses.append(train_loss)
    _, _, val_loss = eval_LWR(model, val_loader, criterion, tsteps) 

    val_losses.append(val_loss)
    if val_loss <= best_loss: 
        best_loss = val_loss 
        best_model = model
        torch.save({"lr": optimizer.param_groups[0]['lr'], "model": model}, "new_result/best3_AutoODE" + str(trial) + ".pt")

    end = time.time()
    print("Epoch:", epoch, "completed in:", (end - start), "s. Training loss:", train_loss, ". Val loss:", val_loss)  
    if (len(train_losses) > 30 and np.mean(val_losses[-5:]) >= np.mean(val_losses[-10:-5])):
        break
    scheduler.step() 
    if epoch % 5 == 0: print(optimizer.param_groups[0]['lr']) 

In [ ]:
tsteps_test = steps_test.shape[0] 
preds, trues, test_loss = test_LWR(model, test_loader, criterion, tsteps_test) 

In [ ]:
test_loss, np.sqrt(test_loss) 

In [ ]:
torch.save({"preds": preds, "trues": trues, "model": model}, "new_result/final_AutoODE3.pt") 

### Training hybrid model 

In [27]:
import time 

for epoch in range(1, num_epoch + 1): 
    start = time.time()
    train_loss = train_hybrid_LWR(model, train_loader, optimizer, criterion, tsteps)[-1]
    train_losses.append(train_loss) 
    print("------Evaluation------") 
#     print(sum(p.numel() for p in model.parameters() if p.requires_grad))
    _, _, val_loss = eval_hybrid_LWR(model, val_loader, criterion, tsteps) 
#     print(sum(p.numel() for p in model.parameters() if p.requires_grad))
    val_losses.append(val_loss)
    if val_loss <= best_loss: 
        best_loss = val_loss 
        best_model = model
        torch.save({"epoch": epoch, "model": best_model}, "time_hybridAutoODE_joint.pt") 
    best_loss = best_loss
    end = time.time()
    print("Epoch:", epoch, "completed in:", (end - start), "s. Training loss:", train_loss, ". Val loss:", val_loss)  
    if (len(train_losses) > 30 and np.mean(val_losses[-5:]) >= np.mean(val_losses[-10:-5])):
        break
    scheduler.step() 
    if epoch % 5 == 0: 
        print(optimizer.param_groups[0]['lr']) 

------Evaluation------
Epoch: 1 completed in: 1463.8357820510864 s. Training loss: 2.0131008537330617 . Val loss: 1.335237555053506
------Evaluation------
Epoch: 2 completed in: 1728.9949719905853 s. Training loss: 1.2595100041001157 . Val loss: 1.1809719815115822
------Evaluation------
Epoch: 3 completed in: 1746.774498462677 s. Training loss: 1.1292362967196916 . Val loss: 1.0796818610495174
------Evaluation------
Epoch: 4 completed in: 1721.362154006958 s. Training loss: 1.0491306896990702 . Val loss: 1.0080065545386303
------Evaluation------
Epoch: 5 completed in: 1738.0617656707764 s. Training loss: 0.9929364823154455 . Val loss: 0.9682416433160266
0.0015475618749999996
------Evaluation------
Epoch: 6 completed in: 1731.2247195243835 s. Training loss: 0.9414823700169369 . Val loss: 0.9171912349257376
------Evaluation------
Epoch: 7 completed in: 1735.9421977996826 s. Training loss: nan . Val loss: nan


KeyboardInterrupt: 

In [ ]:
# trial_1 = torch.load("AutoODE_result/time/time_hybridAutoODE1.pt") 
# LWR_model = trial_1["model"] 
# LWR_model.to(device) 

In [29]:
test_sensors = torch.tensor(np.arange(77))  
test_sensors 

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76], dtype=torch.int32)

In [31]:
tsteps_test = steps_test.shape[0] 
criterion = nn.MSELoss() 
preds, trues, test_loss = test_hybrid_LWR(best_model, test_loader, criterion, test_sensors.long(), tsteps_test) 

nan


KeyboardInterrupt: 

In [ ]:
test_loss, np.sqrt(test_loss) 

In [ ]:
torch.save({"preds": preds, "trues": trues, "model": LWR_model}, "AutoODE_result/time/time_AutoODE7.pt") 